In [1]:
import pandas

In [2]:
df = pandas.read_csv('data/master_df.csv', index_col=0)

In [3]:
df.sample(5)

,field_office,Applications Received2,Approved3,Denied4,Pending5,quarter,state,code,year,fy
217,Providence,786.0,491.0,62.0,1197.0,"FY 2014, 2nd Q, Jan. 1-March 31, 2014",Rhode Island,PRO,2014.0,FY 2014
301,Providence,2656.0,2621.0,269.0,638.0,NaN,Rhode Island,PRO,NaN,FY 2011
37,San Bernardino,4862.0,3682.0,473.0,9695.0,"FY 2016, 4th Q, July 1-Sept. 30, 2016",California,SBD,2016.0,FY 2016
69,Jacksonville,1314.0,1324.0,159.0,2341.0,"FY 2015, 4th Q, July 1-Sept. 30, 2015",Florida,JAC,2015.0,FY 2015
81,West Palm Beach,2295.0,1716.0,345.0,3042.0,"FY 2015, 2nd Q, Jan. 1-March 31, 2015",Florida,WPB,2015.0,FY 2015


In [4]:
code_to_city = {}
code_to_state = {}
city_to_code = {}
for i,entry in df.dropna(subset=['code']).iterrows():
#     print(entry.field_office, entry.code)
    code_to_city[entry.code.strip()] = entry.field_office.strip()
    code_to_state[entry.code.strip()] = entry.state.strip()
    city_to_code[entry.field_office.strip()] = entry.code.strip()
    # discovered a few missing entries
code_to_city['CSC'] = float('nan')
code_to_state['CSC'] = 'California'

code_to_city['NBC'] = 'National Benefits Center'
code_to_state['NBC'] = float('nan')

code_to_city['NSC'] = 'NEBRASKA SERVICE CENTER'
code_to_state['NSC'] = 'Nebraska'

code_to_city['VSC'] = 'VERMONT SERVICE CENTER'
code_to_state['VSC'] = 'Vermont'

code_to_city['TSC'] = 'TEXAS SERVICE CENTER'
code_to_state['TSC'] = 'Texas'

# don't know what this is, and it doesn't have a label
code_to_city['GCU'] = "Garden City"
code_to_state['GCU'] = "New York"

code_to_city['ANC'], code_to_state['ANC']

('Anchorage', 'Alaska')

In [5]:
print(len(df))
df.sample(3)

2963


,field_office,Applications Received2,Approved3,Denied4,Pending5,quarter,state,code,year,fy
75,Oakland Park,2871.0,2558.0,351.0,5996.0,"FY 2015, 1st Q, Oct. 1-Dec. 31, 2014",Florida,OKL,2014.0,FY 2015
187,Long Island,3370.0,3140.0,278.0,9507.0,"FY 2019, 2nd Q, Jan. 1-Mar. 31, 2019",New York,LNY,2019.0,FY 2019
43,San Francisco,8106.0,5024.0,300.0,16579.0,"FY 2016, 3rd Q, April 1-June 30, 2016",California,SFR,2016.0,FY 2016


In [6]:
df.dtypes

field_office               object
Applications Received2    float64
Approved3                 float64
Denied4                   float64
Pending5                  float64
quarter                    object
state                      object
code                       object
year                      float64
fy                         object
dtype: object

In [7]:
sum_df = df.dropna(subset=['code']).groupby(['code', 'fy']).sum()
sum_df

Applications Received2  Approved3  Denied4  Pending5    year
code fy                                                                   
1    FY 2016                     7.0        8.0      0.0       0.0  2016.0
     FY 2017                     0.0        0.0      1.0       0.0  8067.0
     FY 2018                     0.0        0.0      0.0       0.0  8071.0
     FY 2019                     0.0        0.0      0.0       0.0  4037.0
14   FY 2016                    11.0        8.0      0.0       4.0  2016.0
...                              ...        ...      ...       ...     ...
YAK  FY 2016                  2033.0     1515.0    105.0    3059.0  8063.0
     FY 2017                  2280.0     1997.0    225.0    4453.0  8067.0
     FY 2018                  2042.0     2216.0    112.0    3102.0  8071.0
     FY 2019                  1233.0     1385.0     93.0    3319.0  4037.0
     FY 2020                   408.0      562.0     33.0    1924.0     0.0

[985 rows x 5 columns]

In [8]:
del sum_df['year']

In [9]:
sum_df['field_office'] = sum_df.index.map(lambda tup: code_to_city[tup[0]])

In [10]:
sum_df['state'] = sum_df.index.map(lambda tup: code_to_state[tup[0]])

In [11]:
sum_df['nationality'] = sum_df.state.map(
    lambda state: 'international' if state in ['United Kingdom', 'Italy', 'Germany', 'South Korea']
                  else 'US (territory)' if state in ['Guam', 'U.S. Virgin Islands', 'Puerto Rico']
                  else 'US'
)

In [13]:
sum_df[sum_df.nationality == 'US (territory)'].to_csv('data/sum_by_year_territory.csv')
sum_df[sum_df.nationality == 'US'].sort_values(by=['state', 'field_office']).to_csv('data/sum_by_year.csv')
sum_df[sum_df.nationality == 'international'].to_csv('data/sum_by_year_international.csv')

In [15]:
sum_df.loc['IMP']

,Applications Received2,Approved3,Denied4,Pending5,field_office,state,nationality
fy,,,,,,,
FY 2010,762.0,677.0,52.0,297.0,Imperial,California,US
FY 2011,1174.0,963.0,29.0,391.0,Imperial,California,US
FY 2012,3232.0,1642.0,86.0,1817.0,Imperial,California,US
FY 2013,1333.0,1281.0,193.0,2746.0,Imperial,California,US
FY 2014,1435.0,1582.0,251.0,2139.0,Imperial,California,US
FY 2015,1221.0,980.0,212.0,1723.0,Imperial,California,US
FY 2016,1867.0,1214.0,173.0,3021.0,Imperial,California,US
FY 2017,1930.0,1546.0,350.0,4739.0,Imperial,California,US
FY 2018,1504.0,1279.0,376.0,3485.0,Imperial,California,US
